<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U \
langchain \
langchain-community \
langchain-text-splitters \
faiss-cpu \
sentence-transformers \
transformers \
accelerate


In [ ]:
import os

os.makedirs("docs", exist_ok=True)


In [ ]:
with open("docs/register.md", "w", encoding="utf-8") as f:
    f.write("""
# التسجيل في منصة عون

## خطوات التسجيل
- الدخول على موقع عون
- إنشاء حساب جديد
- إدخال البيانات الأساسية
- تأكيد رقم الهاتف

## ملاحظات
- يجب إدخال بيانات صحيحة
- رقم الهاتف إلزامي
""")


In [ ]:
with open("docs/case_submit.md", "w", encoding="utf-8") as f:
    f.write("""
# رفع حالة في منصة عون

## خطوات رفع الحالة
- تسجيل الدخول إلى الحساب
- اختيار رفع حالة جديدة
- إدخال بيانات الأسرة
- إرفاق المستندات المطلوبة
- إرسال الطلب للمراجعة

## ملاحظات
- يجب إرفاق مستندات واضحة
- الطلب يخضع للمراجعة
""")


In [ ]:
with open("docs/terms.md", "w", encoding="utf-8") as f:
    f.write("""
# شروط الاستفادة من منصة عون

- أن تكون البيانات صحيحة
- أن تكون الحالة حقيقية
- تقديم المستندات المطلوبة
- قبول سياسة الاستخدام
""")


In [ ]:
with open("docs/faq.md", "w", encoding="utf-8") as f:
    f.write("""
# الأسئلة الشائعة

## هل التسجيل مجاني؟
نعم، التسجيل مجاني تمامًا.

## هل يمكن تعديل البيانات؟
نعم، يمكن تعديل البيانات قبل إرسال الطلب.
""")


In [ ]:
!ls docs


In [ ]:
from langchain_community.document_loaders import TextLoader

files = [
    "docs/register.md",
    "docs/case_submit.md",
    "docs/terms.md",
    "docs/faq.md"
]

documents = []

for file in files:
    loader = TextLoader(file, encoding="utf-8")
    documents.extend(loader.load())

print("عدد الملفات:", len(documents))


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)

print("عدد القطع:", len(chunks))


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

vector_db = FAISS.from_documents(chunks, embeddings)


In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    task="text-generation",
    model="google/flan-t5-base",
    max_new_tokens=200
)

llm = HuggingFacePipeline(pipeline=pipe)
